# Start the system

In [26]:
#1. Libraries
import psycopg2 as pg
import pandas as pd
import os
import re
import pycountry_convert as pc
import matplotlib.pyplot as plt
import datetime
import math
import numpy as np

### Read csv files

In [27]:
df = pd.read_csv("../data/listings_al.csv")

### Transformation methods

In [28]:
def get_rating(score):
    if score < 88: return "Bellow average"
    if score < 93: return "Average"
    if score < 96: return "Good"
    if score < 99: return "Very good"
    if score < 100: return "Excelent"

In [29]:
def get_accuracy(score):
    if score < 9: return "Accurate description"
    return "Description is not accurate"

In [30]:
def get_cleanliness(score):
    if score < 9: return "Not clean"
    return "Clean"

In [31]:
def get_communication(score):
    if score < 9: return "Bad communication"
    return "Good communication"

In [32]:
def get_location(score):
    if score < 9: return "Bad location"
    return "Good location"

### Create clean dataframe

In [33]:
ratings = [get_rating(s) for s in df['review_scores_rating']]
accuracy = [get_accuracy(s) for s in df['review_scores_accuracy']]
cleanliness = [get_cleanliness(s) for s in df['review_scores_cleanliness']]
communication = [get_communication(s) for s in df['review_scores_communication']]
location = [get_location(s) for s in df['review_scores_location']]

In [34]:
columns = ['rating','accuracy','cleanliness','communication','location']
df_review_dimension = pd.DataFrame(np.stack((ratings, accuracy, cleanliness,communication,
                                          location),axis=-1), columns = columns)
df_review_dimension.shape

(17168, 5)

Remove duplicates:

In [35]:
df_review_dimension = df_review_dimension.drop_duplicates()
df_review_dimension.shape

(63, 5)

Add primary key:

In [36]:
pks = [i for i in range(1,df_review_dimension.shape[0]+1)]
df_review_dimension.insert(0, "review_id", pks, True) 

In [37]:
df_review_dimension

,review_id,rating,accuracy,cleanliness,communication,location
0,1,Very good,Description is not accurate,Clean,Good communication,Good location
1,2,Average,Description is not accurate,Clean,Good communication,Good location
4,3,None,Description is not accurate,Clean,Good communication,Good location
5,4,Good,Description is not accurate,Clean,Good communication,Good location
10,5,Bellow average,Description is not accurate,Clean,Good communication,Good location
...,...,...,...,...,...,...
10397,59,Good,Description is not accurate,Not clean,Good communication,Bad location
10848,60,Good,Accurate description,Clean,Good communication,Bad location
11179,61,None,Accurate description,Not clean,Good communication,Bad location
12953,62,Average,Description is not accurate,Clean,Bad communication,Bad location
